In [ ]:
water_train <- read_csv("train_final_v2.csv")|>
select(NDVI, AWEI, water_index)
head(water_train)

In [1]:
water_recipe <- recipe(water_index ~ ., data = water_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())

water_spec <- nearest_neighbor(weight_func = "rectangular", 
                              neighbors = tune()) |>
  set_engine("kknn") |>
  set_mode("regression")

water_vfold <- vfold_cv(water_train, v = 5, strata = water_index)

water_wkflw <- workflow() |>
  add_recipe(water_recipe) |>
  add_model(water_spec)

water_wkflw

ERROR: Error in step_center(step_scale(recipe(water_index ~ ., data = water_train), : could not find function "step_center"


In [ ]:
gridvals <- tibble(neighbors = seq(from = 180, to = 201, by = 1))

water_results <- water_wkflw |>
  tune_grid(resamples = water_vfold, grid = gridvals) |>
  collect_metrics() |>
  filter(.metric == "rmse")

# show the results
water_results

In [ ]:
# show only the row of minimum RMSPE
water_min <- water_results |>
  filter(mean == min(mean))

water_min

In [2]:
kmin <- water_min |> 
pull(neighbors)

water_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = kmin) |> #kmin
  set_engine("kknn") |>
  set_mode("regression")

water_fit <- workflow() |>
  add_recipe(water_recipe) |>
  add_model(water_spec) |>
  fit(data = water_train)

test_masked_new <- read.csv("test_masked_v2.csv")

water_summary <- water_fit |>
predict(test_masked_new) |>
bind_cols(test_masked_new) |>
rename(water_index = .pred)|>
select(DHSID_EA, water_index)
#metrics(truth = water_index, estimate = .pred) |>
#filter(.metric == 'rmse')
water_summary

ERROR: Error in pull(water_min, neighbors): could not find function "pull"


In [ ]:
wtr_test <- read.csv("water_test_v3.csv")

w_summary <- water_fit |>
  predict(wtr_test) |>
  bind_cols(wtr_test) |>
   metrics(truth = water_index, estimate = .pred) |>
   filter(.metric == 'rmse')
w_summary


In [44]:
write.csv(water_summary, file = "solution3.csv")